In [1]:
import pandas as pd
import math
from skimpy import skim
import xlsxwriter
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer, util
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score, mean_squared_error, r2_score, roc_auc_score, balanced_accuracy_score
from sklearn import preprocessing
from sklearn.feature_selection import mutual_info_classif
from hiclass import LocalClassifierPerParentNode, LocalClassifierPerLevel

import numpy as np
import pickle
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

sns.set()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
ndns_edited = pd.read_csv('../../SFS/NDNS UK/ndns_edited.csv', encoding='unicode_escape')
ndns_edited.loc[ndns_edited['subfoodgroupdesc_edited'].notnull(), 
                'subfoodgroupdesc'] = ndns_edited[ndns_edited['subfoodgroupdesc_edited'].notnull()]['subfoodgroupdesc_edited']
ndns_edited = ndns_edited.drop(['subfoodgroupdesc_edited'], axis=1)

In [3]:
print(ndns_edited['parentcategory'].nunique())
print(ndns_edited['mainfoodgroupdesc'].nunique())
print(ndns_edited['subfoodgroupdesc'].nunique())

17
59
145


In [4]:
predictions = pd.read_csv('../../SFS/NDNS UK/predictions_all_LDA_HI.csv')

In [5]:
predictions[predictions['parentcategory_lab'].isna()].shape

(83247, 14)

In [6]:
to_label = predictions[predictions['parentcategory_lab'].isna()].sample(frac=1, random_state=0).reset_index(drop=True)
# to_label.loc[to_label['product_name'].isna(), 'product_name'] = to_label[to_label['product_name'].isna()]['product_list_name']
to_label = to_label[['product_id', 'product_list_name', 'store', 'parentcategory_pred', 
                     'mainfoodgroup_pred', 'subfoodgroup_pred']].rename(columns={'product_list_name': 'product_name'})

In [7]:
to_label.shape

(83247, 6)

In [9]:
parentcategory = ndns_edited['parentcategory'].unique().tolist()
mainfoodgroup = ndns_edited.groupby('parentcategory')['mainfoodgroupdesc'].agg(['unique']).reset_index().values
subfoodgroup = ndns_edited.groupby('mainfoodgroupdesc')['subfoodgroupdesc'].agg(['unique']).reset_index().values

In [10]:
def write_to_excel(df, fname):
    
    # Write to xlsx file 
    writer = pd.ExcelWriter(fname, engine='xlsxwriter')
    n_rows = df.shape[0]
    df.to_excel(writer, sheet_name='Sheet1', index=False)

    # Assign workbook and worksheet
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    
    # Creation of unlocked format
    unlocked = workbook.add_format({'locked': False})
    worksheet.set_column('A:XDF', None, unlocked)

    worksheet.write('H1', 'parentcategory')
    worksheet.write('I1', 'mainfoodgroupdesc')
    worksheet.write('J1', 'subfoodgroupdesc')

    worksheet2 = workbook.add_worksheet('categories')
    worksheet2.write_column('A2', parentcategory)
    col = 0

    for i in range(len(mainfoodgroup)):
        col += 1
        cat = mainfoodgroup[i]
        worksheet2.write(0, col, cat[0])
        worksheet2.write_column(1, col, cat[1].tolist())

    for i in range(len(subfoodgroup)):
        col += 1
        cat = subfoodgroup[i]
        worksheet2.write(0, col, cat[0])
        worksheet2.write_column(1, col, cat[1].tolist())

    # Creation of the dropdown menus
    worksheet.data_validation('H2:H'+str(1+n_rows), {'validate' : 'list', 'source': '=categories!$A$2:$A$'+str(1+len(parentcategory))})
    for i in range(n_rows):
        worksheet.data_validation('I'+str(2+i), {'validate' : 'list', 'source': '=INDEX(categories!$B$2:$R$12, 0, MATCH($H$'+str(2+i)+', categories!$B$1:$R$1, 0))'})
    for i in range(n_rows):
        worksheet.data_validation('J'+str(2+i), {'validate' : 'list', 'source': '=INDEX(categories!$S$2:$BY$16, 0, MATCH($I$'+str(2+i)+', categories!$S$1:$BY$1, 0))'})
    
    # Close the workbook
    workbook.close()

In [11]:
for i in range(2, math.ceil(83248/500)):
    if i==math.ceil(83248/500)-1:
        df = to_label.iloc[i*500:,:]
    else:
        df = to_label.iloc[i*500:(i+1)*500,:]
    fname = f'../../SFS/NDNS UK/labelling_exercise/sample_{i+1}.xlsx'
    write_to_excel(df, fname)